# CSVファイルの作成例

Sample notebook to create csv files from WIN data

|Date   |Commit history|
|:------|:-------------|
|04/05 2021|First created by Yasunori Sawaki (Kyoto Univ.)|
|04/28     |全時間データの作成|

To run each cell, only to enter `Shift+Enter` (cf. `Ctrl+Enter`)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install `ObsPy`



In [ ]:
#@markdown - <u>このセルを実行</u>してください
#@markdown - 出力結果は致命的なエラーでない限り，無視して問題ない  
#@markdown - `'Obspy installation completed!'`が出ればインストール完了

_if_obspy = !pip list | grep obspy

if not _if_obspy:
    !pip uninstall scipy -y -q  # !pip install --upgrade scipy
    !pip install obspy -q 
    print('Obspy installation completed!')

## Import required modules

Pythonでは，パッケージを`import`することで使用可能になる

In [ ]:
import sys
import os
from pathlib import Path
import argparse
import requests

import numpy as np
import matplotlib.pyplot as plt

import obspy as ob

notebookname = requests.get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
print('notebookname ->', notebookname)

### パス設定

1. 左側のタブ`ファイル`から`EarthScienceExperiment`という名前のフォルダを探す
2. フォルダ上で右クリックして「パスをコピー」を選択
3. 右の`projectBaseDir`パスを貼り付ける（セルは自動的に更新される）

Look for the directory named `EarthScienceExperiment` and right click for **`Copy path`**.

In [13]:
#@title Paste directory path / ↓パスを貼り付ける↓ {run: "auto"}

projectBaseDir = '' #@param {type:"string"}
projectBaseDir = Path(projectBaseDir)

#@markdown 例：projectBaseDir : `"/content/drive/MyDrive/EarthScienceExperiment"`

if (projectBaseDir/notebookname).exists():
    if not str(projectBaseDir) in sys.path:
        sys.path.append(str(projectBaseDir))
        from winpy import WinTools, save_as_csv
else:
    raise FileNotFoundError(f'Path not found. Check again projectBaseDir: {projectBaseDir}')

os.chdir(projectBaseDir)
print(f'Current dir: {projectBaseDir}')

Current dir: .


## `WinTools` クラスの呼び出し

In [ ]:
wt = WinTools()

### Read WIN files

In [ ]:
st = wt.read(projectBaseDir/'data'/'2021042015.9905100001')

### Plot waveforms

In [ ]:
st.plot();  # wt.stream.plot();

### Customize ploting styles

- [Waveform Plotting Tutorial](https://docs.obspy.org/tutorial/code_snippets/waveform_plotting_tutorial.html) (`ObsPy` HP)

In [ ]:
starttime = st[0].stats.starttime
st[0].plot(
    starttime=starttime+730, endtime=starttime+780,
    color='blue', linewidth=0.9, size=(800, 300)
);

### Save waveforms as various formats

In [ ]:
## CSV - Comma Separated Values
wt.write(projectBaseDir/'output'/'test.csv', 'CSV', decimate=10)

## SAC - Seicmic Analysis Code
# wt.write(projectBaseDir/'output'/'test.sac', 'SAC')

In [ ]:
## ASCII - American Standard Code for Information Interchange
wt.write(projectBaseDir/'output'/'test.txt', 'TSPAIR', decimate=10)

## 応用編

### ファイル名を自動で調整

In [ ]:
filepath = projectBaseDir/'data'/'2021042015.9905100001'

print('filepath ->', filepath)
print('filename  :', filepath.name)
print('filename2 :', filepath.name.split(".")[0])

st = wt.read(filepath)

In [ ]:
outputfilepath = projectBaseDir/'output'/f'{filepath.name.split(".")[0]}.csv'

print(outputfilepath)

## Save as CSV
wt.write(outputfilepath, 'CSV', decimate=10)

### 全期間データの作成

1時間WINファイルを全て選択し，結合する

In [ ]:
allfilenames = set((projectBaseDir/'data').glob('*.9905100001'))
allfilenames

In [ ]:
## Create an empty stream
st = ob.Stream()

for fname in allfilenames:
    print('Now :', fname.name)

    ## add streams
    st += wt.read(fname)

## Merge stream
st.merge()

開始日時を抜き出して，ファイル名にする

- `winpy.save_as_csv()`を使って，CSV変換を行う

In [ ]:
outputfilename = st[0].stats.starttime.strftime('%Y%m%d') + '_all.csv'
print(outputfilename)

## Save as CSV
save_as_csv(projectBaseDir/'output'/outputfilename, st, decimate=10)